Доверительный инервал - это интервал, построенный с помощью случайной выборки из распределения с неизвестным параметром, такой, что он содержит данный параметр с заданной вероятностью.

При условии что для выборки Х с нормальным распределением $N(m, \sigma)$. Математическое ожидание равно m. При n независимых измерениях, критерий дисперсии D(x) = $\sigma^2 \over n$

По определению t-статистики, величина t = ${<X> - m} \over \sqrt{\sigma^2 \over n}$. Данная величина имеет нормальное распределение N(0,1).

При введении достоверности p = 1 -$\alpha$ , вероятность нахождения величины t определить как $P(-t_{1-{\alpha \over 2}} <= t <= t_{1-{\alpha \over 2}}) = 1 - \alpha$ в силу симметрии. Где t - квантиль нормального распределения

Таким образом для заданного значения m получаем доверительный интервал $<X> - t_{1-{\alpha \over 2}}  {\sigma \over \sqrt n} \le m \le <X> + t_{1-{\alpha \over 2}} {\sigma \over \sqrt n} $,

при условии что известна $\sigma$.

В случае, если $\sigma$ неизвестна, вместо нормального распределения можно воспользоваться распределением Стьюдента $t_{1-{\alpha \over 2}}(n-1)$

__1.__ 

Известно, что генеральная совокупность распределена нормально
со средним квадратическим отклонением, равным 16.

Найти доверительный интервал для оценки математического ожидания a с надежностью 0.95,
если выборочная средняя M = 80, а объем выборки n = 256.

In [89]:
import numpy as np
import scipy.integrate as integrate
from scipy import stats

In [90]:
def norm(x, sigma, a):
    A = 1/(sigma * np.sqrt(np.pi * 2))
    B = (x-a)**2 / (2 * sigma**2)
    
    result = A * np.exp(-B)
    return result

In [91]:
def student(x, n):
    a = n - 1
    b = n - 2

    A = n - 1
    B = n - 2
    while a > 1 and b > 1:
        a -= 2
        b -= 2
        if a >= 1:
            A *= a
        if b >= 1:
            B *= b
    if n % 2 == 0:
        result = A / (2 * np.sqrt(n) * B)
    else:
        result = A / (np.pi * np.sqrt(n) * B)

    value = np.math.pow((1 + x ** 2 / n), -(n + 1) / 2)

    return result * value
        

In [92]:
def get_quantil(f_x, alpha, e=0.001):
    p = alpha
    start = -np.inf
    if alpha < 0.5:
        start_point = -10
        end = 0
    else:
        start_point = 0
        end = 10

    point = (start_point + end) / 2
    while True:
        integral = integrate.quad(f_x, start, point)[0]
        if abs(integral - p) < e:
            break
        elif integral > p:
            end = point
            point = (end + start_point) / 2
        elif integral < p:
            start_point = point
            point = (end + start_point) / 2

    return point

In [93]:
def get_interval_value(sigma, quantil, n):
    return quantil * sigma / np.sqrt(n)

Для нормального распределения можно взять половину значения $\alpha$

In [94]:
p = 0.95
alpha = 1-p
alpha_2 = alpha / 2
print(alpha_2)

0.025000000000000022


In [95]:
# for current alpha can be calculated quantil
result = get_quantil(alpha=alpha_2, f_x = lambda x: norm(x, 1, 0))

In [96]:
print(result)

-1.953125


$\sigma$ = 16 , n = 256, $\bar{X} = 80$

In [97]:
M = 80
sigma = 16
n = 256
interval = get_interval_value(sigma=sigma, quantil=abs(result), n=n)
left_x = M - interval
right_x = M + interval

In [98]:
#Наш доверительный интервал с вероятностью 95%
print(left_x, right_x)

78.046875 81.953125


__2.__
В результате 10 независимых измерений некоторой величины X, выполненных с одинаковой точностью,
получены опытные данные:

6.9, 6.1, 6.2, 6.8, 7.5, 6.3, 6.4, 6.9, 6.7, 6.1

Предполагая, что результаты измерений подчинены нормальному закону распределения вероятностей,

оценить истинное значение величины X при помощи доверительного интервала, покрывающего это

значение с доверительной вероятностью 0,95.

Check quantil for student distribution

In [99]:
alpha_2 = 0.025
p = 1 - alpha_2
k = 9

In [100]:
result = get_quantil(alpha=p, f_x = lambda x: student(x, k))

In [101]:
print(result, f",from table: {2.262}")

2.265625 ,from table: 2.262


Так как $\sigma$ неизвестна то вместо нее используем несмещенную оценку дисперии и вместо нормального распределения используем квантили распределения стьдента

In [107]:
data = np.asarray([6.9, 6.1, 6.2, 6.8, 7.5, 6.3, 6.4, 6.9, 6.7, 6.1])
n = 10

In [108]:
X_avr = stats.tmean(data)
stddev = stats.tstd(data)

In [109]:
print(X_avr, stddev) 

6.590000000000001 0.4508017549014448


In [110]:
p = 0.95

Для этого значения вычислим квантиль стьюдента

In [111]:
st_quantil = get_quantil(alpha=p, f_x = lambda x: student(x, n=n-1))

In [112]:
print(st_quantil)

1.8359375


In [113]:
interval = get_interval_value(sigma=stddev, n=n, quantil=abs(st_quantil))

In [114]:
left_x = X_avr - interval
right_x = X_avr + interval

In [116]:
#Наш доверительный интервал с вероятностью 95%
print(left_x, right_x)

6.328276035240593 6.851723964759408


__3.__

Утверждается, что шарики для подшипников, изготовленные автоматическим станком, имеют средний диаметр 17 мм.

Используя односторонний критерий с α=0,05, проверить эту гипотезу, если в выборке из n=100 шариков средний диаметр

оказался равным 17.5 мм, а дисперсия известна и равна 4 мм.

In [117]:
alpha = 0.05
p = 1 - alpha
d_0 = 17
d_1 = 17.5
D_x = 4
n = 100

In [118]:
sigma = np.sqrt(D_x)

Дисперсия известна значит можно использовать нормальное распределение

По определению случайной величины t

$t = {{\bar{X} - M(x)} \over {\sigma \over \sqrt n}}$

По умолчанию М(х) = d_0 , а $\bar{X}$ = d_1

In [119]:
t_real = (d_1 - d_0) / (sigma / np.sqrt(n))

С другой стороны может быть определена как квантиль нормального распределения с достоверностью р

In [120]:
t_theory = get_quantil(alpha=p, f_x = lambda x: norm(x, 1, 0))

In [121]:
print(t_theory, t_real)

1.640625 2.5


t_real > t_theory => Изначальное предположение что d_0 истина - неверно

__4.__

Продавец утверждает, что средний вес пачки печенья составляет 200 г.

Из партии извлечена выборка из 10 пачек. Вес каждой пачки составляет:

202, 203, 199, 197, 195, 201, 200, 204, 194, 190.

Известно, что их веса распределены нормально.

Верно ли утверждение продавца, если учитывать, что доверительная вероятность равна 99%?

Предварительный анализ неизвестен => Используем распределение стьюдента вместо нормаьного распределения

In [123]:
data = np.asarray([202, 203, 199, 197, 195, 201, 200, 204, 194, 190])
n = 10
d_0 = 200
p = 0.99

In [124]:
X_avr = stats.tmean(data)
stddev = stats.tstd(data)

In [128]:
t_real = abs(X_avr - d_0) / (stddev / np.sqrt(n))

In [129]:
t_theory = get_quantil(alpha=p, f_x = lambda x: student(x, n-1))

In [130]:
print(t_theory, t_real)

2.8125 1.0651074037450896


t_real < t_theory